[View in Colaboratory](https://colab.research.google.com/github/seonhonam/RC/blob/master/ESN.ipynb)

In [0]:
import tensorflow as tf
import numpy as np
import networkx as nx

In [0]:
n_inputs = 100
n_reservoir = n_inputs*100
n_readout_inputs = n_inputs*10
n_readout_hidden1 = 400
n_readout_hidden2 = 400
n_readout_hidden3 = 200
n_readout_outputs = 100
n_steps = 10

In [0]:
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs], name = "X")
y = tf.placeholder(tf.float32, [None, n_steps, n_readout_outputs], name = "y")

In [0]:
#reservoir layer를 만들어 준다
def reservoir_layer(X,n_reservoir,internal_network, before_reservoir_state,
                   name):
  with tf.name_scope("name"):
    n_inputs = int(X.get_shape()[1])
    stddev = 2/np.sqrt(n_inputs + n_reservoir)
    W_in = tf.constant(tf.truncated_normal((n_inputs, n_resevoir),
                                           stddev = stddev))
    W = tf.constant(internal_network)
    if before_reservoir_state is None:
      Z = tf.matmul(X, W_in)
    else:
      Z = tf.matmul(X, W_in)+tf.matmul(before_reservoir_state, W)
    return Z

In [0]:
# network에 들어갈 adjacency matrix를 만든다
#ER Random network의 adjacency matirx
def ERN(n_node,p):
  g = nx.erdos_renyi_graph(n_node,p,seed=2,directed=True)
  adm = np.array(nx.adj_matrix(g).todense())
  return adm